# 3 - Data preprocessing

I tried to be minimalistic. While geohash6 value is converted to distance away from certain point of interests and datetime are engineered into sin/cos equivalent. I try to be logical here and use only previous 2 hours demands and previous 7 days demands of the predicted time window +/- 1 hour.

### Data preprocessing and feature engineerings
- distance from hand-picked POI - qp09d8, qp03xx, qp03wf
- day - day_of_week_sin, day_of_week_cos
- hour - hour_sin, hour_cos
- fiteen minute - fifteen_minute_sin, fifteen_minute_cos
- last two hours demand - demand_t0, demand_t1, ... demand_t7
- last seven days demands of the predicted time +/- 1 hour - demand_{day}d_t{ith}
- next 5 demands (target_1, ... target_5)

## NOTE I HAVE INCLUDED MORE FEATURES THAN MY MODEL ACTUALLY NEEDED AS I WAS TOYING AND EXPERIMENTING WHAT ARE THE BEST FEATURES TO REDUCE COMPLEXITY OF MY MODEL WHILE MAINTAINING A GOOD RMSE/SMAPE

In [2]:
import numpy as np
import pandas as pd

import geohash
import gc
from geopy import distance

In [3]:
EVAL_DAY_ONWARDS = 50

In [4]:
df = pd.read_pickle('../data/0-df.pkl')
train_df = df[df.day <= EVAL_DAY_ONWARDS]
eval_df = df[df.day > EVAL_DAY_ONWARDS]

In [5]:
def timestamp_to_hour_minute(timestamp):
    timestamp = timestamp.split(':')
    return int(timestamp[0]), int(timestamp[1])

def get_distance_by_gh(gh1, gh2):
    coor1 = geohash.decode(gh1)
    coor2 = geohash.decode(gh2)
    return distance.distance(coor1, coor2).km

In [6]:
train_df['hour'] = train_df.timestamp.apply(lambda x: timestamp_to_hour_minute(x)[0])
train_df['hour_sin'] = np.sin(train_df.hour*(np.pi/24))
train_df['hour_cos'] = np.cos(train_df.hour*(np.pi/24))

eval_df['hour'] = eval_df.timestamp.apply(lambda x: timestamp_to_hour_minute(x)[0])
eval_df['hour_sin'] = np.sin(eval_df.hour*(np.pi/24))
eval_df['hour_cos'] = np.cos(eval_df.hour*(np.pi/24))

/anaconda3/envs/grab/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/envs/grab/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/grab/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [7]:
train_df['day_mod_seven'] = train_df.day.values % 7
train_df['day_of_week_sin'] = np.sin(train_df.day_mod_seven*(np.pi/7))
train_df['day_of_week_cos'] = np.cos(train_df.day_mod_seven*(np.pi/7))

eval_df['day_mod_seven'] = eval_df.day.values % 7
eval_df['day_of_week_sin'] = np.sin(eval_df.day_mod_seven*(np.pi/7))
eval_df['day_of_week_cos'] = np.cos(eval_df.day_mod_seven*(np.pi/7))

/anaconda3/envs/grab/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/envs/grab/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/grab/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [8]:
train_df['per_fifteen_minutes'] = train_df.order.values % 96
train_df['fifteen_minute_sin'] = np.sin(train_df.per_fifteen_minutes*(np.pi/96))
train_df['fifteen_minute_cos'] = np.cos(train_df.per_fifteen_minutes*(np.pi/96))

eval_df['per_fifteen_minutes'] = eval_df.order.values % 96
eval_df['fifteen_minute_sin'] = np.sin(eval_df.per_fifteen_minutes*(np.pi/96))
eval_df['fifteen_minute_cos'] = np.cos(eval_df.per_fifteen_minutes*(np.pi/96))

/anaconda3/envs/grab/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/envs/grab/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/grab/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

### The following parts are to get aggregated values to our model, however they are not making my model better after few trials

In [9]:
# agg_dict = {}
# for gh in train_df.geohash6.unique():
#     temp_df = train_df[train_df.geohash6 == gh]
    
#     agg_dict[gh] = {}
    
#     agg_dict[gh]['per_fifteen_minutes_median'] = temp_df.groupby('per_fifteen_minutes').median().demand.values
#     agg_dict[gh]['per_fifteen_minutes_mean'] = temp_df.groupby('per_fifteen_minutes').mean().demand.values
    
#     agg_dict[gh]['hour_median'] = temp_df.groupby('hour').median().demand.values
#     agg_dict[gh]['hour_mean'] = temp_df.groupby('hour').median().demand.values
    
#     agg_dict[gh]['day_mod_seven_median'] = temp_df.groupby('day_mod_seven').median().demand.values
#     agg_dict[gh]['day_mod_seven_mean'] = temp_df.groupby('day_mod_seven').mean().demand.values

In [10]:
# per fifteen_minutes (of that geohash)
# train_df['per_fifteen_minutes_median'] = (
#     train_df.apply(lambda x: agg_dict[x.geohash6]['per_fifteen_minutes_median'][x.per_fifteen_minutes], axis=1)
# )
# train_df['per_fifteen_minutes_mean'] = (
#     train_df.apply(lambda x: agg_dict[x.geohash6]['per_fifteen_minutes_mean'][x.per_fifteen_minutes], axis=1)
# )

# eval_df['per_fifteen_minutes_median'] = (
#     eval_df.apply(lambda x: agg_dict[x.geohash6]['per_fifteen_minutes_median'][x.per_fifteen_minutes], axis=1)
# )
# eval_df['per_fifteen_minutes_mean'] = (
#     eval_df.apply(lambda x: agg_dict[x.geohash6]['per_fifteen_minutes_mean'][x.per_fifteen_minutes], axis=1)
# )

In [11]:
# # median by hour (of that geohash)
# train_df['hour_median'] = (
#     train_df.apply(lambda x: agg_dict[x.geohash6]['hour_median'][x.hour], axis=1)
# )
# train_df['hour_mean'] = (
#     train_df.apply(lambda x: agg_dict[x.geohash6]['hour_mean'][x.hour], axis=1)
# )

# eval_df['hour_median'] = (
#     eval_df.apply(lambda x: agg_dict[x.geohash6]['hour_median'][x.hour], axis=1)
# )
# eval_df['hour_mean'] = (
#     eval_df.apply(lambda x: agg_dict[x.geohash6]['hour_mean'][x.hour], axis=1)
# )

In [12]:
# # median by day_mod_seven (of that geohash)
# train_df['day_of_week_median'] = (
#     train_df.apply(lambda x: agg_dict[x.geohash6]['day_mod_seven_median'][x.day_mod_seven], axis=1)
# )
# train_df['day_of_week_mean'] = (
#     train_df.apply(lambda x: agg_dict[x.geohash6]['day_mod_seven_mean'][x.day_mod_seven], axis=1)
# )

# eval_df['day_of_week_median'] = (
#     eval_df.apply(lambda x: agg_dict[x.geohash6]['day_mod_seven_median'][x.day_mod_seven], axis=1)
# )
# eval_df['day_of_week_mean'] = (
#     eval_df.apply(lambda x: agg_dict[x.geohash6]['day_mod_seven_mean'][x.day_mod_seven], axis=1)
# )

In [13]:
# train_df['lat'] = train_df.geohash6.apply(lambda x: geohash.decode(x)[0])
# train_df['long'] = train_df.geohash6.apply(lambda x: geohash.decode(x)[1])

# eval_df['lat'] = eval_df.geohash6.apply(lambda x: geohash.decode(x)[0])
# eval_df['long'] = eval_df.geohash6.apply(lambda x: geohash.decode(x)[1])

In [15]:
POI_GEOHASH = ['qp09d8', 'qp03xx', 'qp03wf']
poi_distance_dict = {}
for poi in POI_GEOHASH:
    poi_distance_dict[poi] = {}
    for gh in train_df.geohash6.unique():
        poi_distance_dict[poi][gh] = get_distance_by_gh(poi, gh)

In [16]:
for gh in POI_GEOHASH:
    train_df['distance_from_{0}'.format(gh)] = train_df.geohash6.apply(lambda x: poi_distance_dict[gh][x])
    eval_df['distance_from_{0}'.format(gh)] = eval_df.geohash6.apply(lambda x: poi_distance_dict[gh][x])

/anaconda3/envs/grab/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/grab/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
# NORMALIZE
for gh in POI_GEOHASH:
    gh_key = 'distance_from_{0}'.format(gh)
    gh_mean = train_df[gh_key].mean()
    gh_std = train_df[gh_key].std()
    train_df[gh_key] = (train_df[gh_key] - gh_mean)/gh_std
    eval_df[gh_key] = (eval_df[gh_key] - gh_mean)/gh_std

/anaconda3/envs/grab/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/grab/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [21]:
gh_mean, gh_std

(17.842158336193567, 9.672254418484291)

In [18]:
train_df.to_pickle('../data/preprocessed_train_df.pkl')
eval_df.to_pickle('../data/preprocessed_eval_df.pkl')

### This is the hard work to transform data into tabular

In [19]:
df = train_df.append(eval_df)

In [30]:
csv_file = open('../data/preprocessed_df_for_training.csv', 'w')

headers = [
    'order', 'geohash6', 'day',
    'distance_from_qp09d8', 'distance_from_qp03xx', 'distance_from_qp03wf',
    'target_1', 'target_2', 'target_3', 'target_4', 'target_5',
    'demand_t0', 'demand_t1', 'demand_t2', 'demand_t3', 'demand_t4', 'demand_t5', 'demand_t6', 'demand_t7',
    'day_of_week_sin', 'day_of_week_cos',
    'hour_sin', 'hour_cos',
    'fifteen_minute_sin', 'fifteen_minute_cos',
]

for d in range(1, 8):
    for i in range(13):
        headers.append('demand_{0}d_t{1}'.format(d, i))

csv_file.write(','.join(headers))
csv_file.write('\n')

for i, gh in enumerate(df.geohash6.unique()):
    # print(i, len(df.geohash6.unique()))
    if i % 100 == 0: print(i)
    temp_df = df[df.geohash6 == gh]
    temp_df.set_index('order', inplace=True)
    start_row = temp_df.index[0]
    end_rows = temp_df.index[-1]
    for i in range(start_row + 96*7 + 3, end_rows - 5):
        # put in whatever data needed into here
        arr = {
            'order': i,
            'geohash6': gh,
            'day': temp_df.at[i, 'day'],
            
            'distance_from_qp09d8': temp_df.at[i, 'distance_from_qp09d8'],
            'distance_from_qp03xx': temp_df.at[i, 'distance_from_qp03xx'],
            'distance_from_qp03wf': temp_df.at[i, 'distance_from_qp03wf'],
            
            # LABELS
            'target_1': temp_df.at[i+1, 'demand'],
            'target_2': temp_df.at[i+2, 'demand'],
            'target_3': temp_df.at[i+3, 'demand'],
            'target_4': temp_df.at[i+4, 'demand'],
            'target_5': temp_df.at[i+5, 'demand'],
            
            # Last two hours demands
            'demand_t0': temp_df.at[i, 'demand'],
            'demand_t1': temp_df.at[i-1, 'demand'],
            'demand_t2': temp_df.at[i-2, 'demand'],
            'demand_t3': temp_df.at[i-3, 'demand'],
            'demand_t4': temp_df.at[i-4, 'demand'],
            'demand_t5': temp_df.at[i-5, 'demand'],
            'demand_t6': temp_df.at[i-6, 'demand'],
            'demand_t7': temp_df.at[i-7, 'demand'],
            
            'day_of_week_sin': temp_df.at[i, 'day_of_week_sin'],
            'day_of_week_cos': temp_df.at[i, 'day_of_week_cos'],
            
            'hour_sin': temp_df.at[i, 'hour_sin'],
            'hour_cos': temp_df.at[i, 'hour_cos'],
            
            'fifteen_minute_sin': temp_df.at[i, 'fifteen_minute_sin'],
            'fifteen_minute_cos': temp_df.at[i, 'fifteen_minute_cos'],
        }
        
        for d in range(1, 8):
            for j in range(13):
                arr['demand_{0}d_t{1}'.format(d, j)] = temp_df.at[i + j - 96*d - 3, 'demand']
        
        content = []
        for h in headers:
            content.append(str(arr[h]))
        csv_file.write(','.join(content))
        csv_file.write('\n')
    gc.collect()
    
csv_file.close()

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300


In [31]:
df = pd.read_csv('../data/preprocessed_df_for_training.csv')
df.to_pickle('../data/preprocessed_df_for_training.pkl')
df[df.day <= 50].to_pickle('../data/preprocessed_train_df_for_training.pkl')
df[df.day > 50].to_pickle('../data/preprocessed_eval_df_for_training.pkl')